股票测试

In [ ]:
import requests
import pandas as pd
import json

# DeepSeek API配置
API_URL = "https://api.deepseek.com/v1/chat/completions"
API_KEY = "sk-68a0ce2c2b6b4670b1d03afb583802aa"

# 获取用户输入的参考指标
def get_input():
    gold_price = input("请输入当前金价（美元/盎司）：")
    oil_price = input("请输入当前油价（美元/桶）：")
    news_title = input("请输入最近的财经新闻标题：")
    return gold_price, oil_price, news_title

# 生成DeepSeek API请求

def get_stock_predictions(gold_price, oil_price, news_title):
    prompt = f"""
    根据以下输入，预测A股市场中各板块的走势，并推荐表现最好的个股：
    - 金价：{gold_price} 美元/盎司
    - 油价：{oil_price} 美元/桶
    - 新闻标题：{news_title}

    请按以下格式输出：
    1. 预测主要A股板块的涨跌趋势（如：新能源、消费、医药、科技等）。
    2. 每个推荐板块中推荐3只股票，给出股票代码、公司名称和推荐理由。
    3. 说明分析依据。
    """

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

    data = {
        "model": "deepseek-chat",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post(API_URL, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        print("API请求失败，状态码：", response.status_code)
        print(response.text)
        return None

# 解析API返回结果
def parse_prediction(prediction):
    lines = prediction.split("\n")
    board_trends = []
    stock_recommendations = []

    for line in lines:
        if "板块" in line and "涨" in line or "跌" in line:
            board_trends.append(line)
        elif "股票" in line or "代码" in line:
            stock_recommendations.append(line)

    return board_trends, stock_recommendations

# 主程序
if __name__ == "__main__":
    gold_price, oil_price, news_title = get_input()
    prediction = get_stock_predictions(gold_price, oil_price, news_title)

    if prediction:
        print("\n🔍 DeepSeek 预测结果：")
        board_trends, stock_recommendations = parse_prediction(prediction)

        print("\n📊 A股板块预测：")
        for trend in board_trends:
            print(trend)

        print("\n💡 推荐个股：")
        for stock in stock_recommendations:
            print(stock)
    else:
        print("未能获取预测结果，请稍后再试。")


API请求失败，状态码： 404
{
  "error_msg": "Not Found. Please check the configuration."
}
未能获取预测结果，请稍后再试。


In [ ]:
from openai import OpenAI

# DeepSeek API 配置
client = OpenAI(api_key="sk-68a0ce2c2b6b4670b1d03afb583802aa", base_url="https://api.deepseek.com")

def get_stock_predictions(gold_price, oil_price, news_title):
    with open(prompt_file, "r", encoding="utf-8") as file:
        prompt = file.read()

    prompt = prompt + f"""    
    请按以下格式输出：
    1. 预测主要A股板块的涨跌趋势。
    2. 每个推荐板块中推荐3只股票，给出股票代码、公司名称和推荐理由。
    3. 说明分析依据。
    """

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": "You are a helpful stock prediction assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature = 0.3
        stream=True
    )


    print("\n🔍 DeepSeek 预测结果：")
    for chunk in response:
        if chunk.choices[0].delta and chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)



# 主程序
if __name__ == "__main__":
    prompt_file = input("请输入包含提示词的txt文件路径：")
    prediction = get_stock_predictions(gold_price, oil_price, news_title)

    if prediction:
        print("\n🔍 DeepSeek 预测结果：")
        print(prediction)
    else:
        print("未能获取预测结果，请稍后再试。")


🔍 DeepSeek 预测结果：
1. 预测主要A股板块涨跌趋势  
- **新能源**：受益于油价高位运行（85美元/桶）强化替代能源逻辑，叠加政策支持，预计领涨。  
- **科技（AI/算力）**：新闻事件"deepseek大火"指向AI大模型领域突破，算力基础设施、应用端或迎催化。  
- **消费/医药**：受避险情绪（金价2957美元暗示市场风险偏好波动）压制，短期或震荡。  

2. 推荐个股  
**（1）新能源板块**  
- **300750 宁德时代**：全球动力电池龙头，高油价加速电动车渗透，技术迭代巩固护城河。  
- **601012 隆基绿能**：光伏组件领军企业，受益能源转型与海外需求增长，N型TOPCon量产领先。  
- **300274 阳光电源**：光储逆变器全球市占率第一，海外户储需求回暖+大储放量。  

**（2）科技（AI算力）板块**  
- **300308 中际旭创**：全球光模块龙头，800G产品独占英伟达供应，直接受益AI算力基建。  
- **002230 科大讯飞**：国产大模型第一梯队，"deepseek"竞品催化国产AI应用商业化预期。  
- **688111 金山办公**：WPS深度集成AI功能，党政/企业端信创替代加速。  

3. 分析依据  
- **油价传导逻辑**：布油85美元高位强化新能源经济性，光伏装机旺季+电动车价格战缓和，产业链盈利预期修复。  
- **事件驱动**："deepseek大火"反映AI大模型竞争白热化，算力（硬件）、数据（训练）、应用（办公/教育）全链条受益。  
- **避险情绪边际影响**：金价高位反映地缘风险，但国内稳增长政策对冲下，成长赛道（新能源/AI）仍为资金主攻方向。未能获取预测结果，请稍后再试。
